# M5 XGBoost

After having the data to build our XGBoost model, let's start building a baseline and predict the validation data (in our case is test data).

# Feature Engineering

To prepare for the transform step and improve machine learning accuracy since algorithms tend to have a hard time dealing with high cardinality columns, we define target encoding with smoothing function. The `min_samples_leaf` define a threshold where prior and target mean (for a given category value) have the same weight. Below the threshold prior becomes more important and above mean becomes more important. How weight behaves against value counts is controlled by smoothing parameter

In [15]:
def add_noise(series, noise_level):
    return series * (1 + noise_level * np.random.randn(len(series)))

def target_encode(trn_series=None,  
                  target=None, 
                  min_samples_leaf=100, 
                  smoothing=10,
                  noise_level=0.01):
    
    assert len(trn_series) == len(target)
    temp = pd.concat([trn_series, target], axis=1)
    
    # Compute target mean 
    averages = temp.groupby(by=trn_series.name)[target.name].agg(["mean", "count"])
    # Compute smoothing
    smoothing = 1 / (1 + np.exp(-(averages["count"] - min_samples_leaf) / smoothing))
    # Apply average function to all target data
    prior = target.mean()
    # The bigger the count the less full_avg is taken into account
    averages[target.name] = prior * (1 - smoothing) + averages["mean"] * smoothing
    averages.drop(["mean", "count"], axis=1, inplace=True)
    
    # Apply averages to trn and tst series
    ft_trn_series = pd.merge(
        trn_series.to_frame(trn_series.name),
        averages.reset_index().rename(columns={'index': target.name, target.name: 'average'}),
        on=trn_series.name,
        how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)
    # pd.merge does not keep the index so restore it
    ft_trn_series.index = trn_series.index 
    return add_noise(ft_trn_series, noise_level)  # add_noise(ft_tst_series, noise_level)

We define a transform function to transform features to computer-readable format by combining both label encoding and target encoding. The encoding will be appropriately apply for suitable features to avoid overfitting.

In [18]:
def transform(data):
    
    nan_features = ['event_name_1', 'event_type_1', 'event_name_2', 'event_type_2']
    for feature in nan_features:
        data[feature].fillna('unknown', inplace = True)
        
    #we have many missing value in nan_features which is record under the same category 'unknown'
    #It is more appropriate to use target encoding instead of label encording   
    cat_label = ['id','event_name_1', 'event_type_1', 'event_name_2', 'event_type_2']
    for feature in cat_label: 
        #Encode target labels with value between 0 and n_classes-1 
        encoder = preprocessing.LabelEncoder() 
        #Fit label encoder and return encoded labels
        data[feature] = encoder.fit_transform(data[feature]) 
    
    cat_target = ['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']
    for feature in cat_target: 
        data[feature] = target_encode(data[feature], target=data.demand)
        
    data = reduce_mem_usage(data)
    
    return data

In [19]:
data = transform(data)

Mem. usage decreased to 2483.02 Mb (44.7% reduction)


Let's create list of features.

In [20]:
def simple_fe(data):
    
    # rolling demand features
    data['lag_t28'] = data.groupby(['id'])['demand'].transform(lambda x: x.shift(28))
    data['lag_t29'] = data.groupby(['id'])['demand'].transform(lambda x: x.shift(29))
    data['lag_t30'] = data.groupby(['id'])['demand'].transform(lambda x: x.shift(30))

    # compute rolling mean or moving average
    data['rolling_mean_t7'] = data.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(7).mean())
    data['rolling_std_t7'] = data.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(7).std())
    data['rolling_mean_t30'] = data.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(30).mean())
    data['rolling_mean_t90'] = data.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(90).mean())
    data['rolling_mean_t180'] = data.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(180).mean())
    data['rolling_std_t30'] = data.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(30).std())
    data['rolling_skew_t30'] = data.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(30).skew())
    data['rolling_kurt_t30'] = data.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(30).kurt())
    
    
    # price features: using lag of 1 week (t7), 4 weeks(t28), 12 weeks(t84), 24 weeks (t168), 52 weeks (t365) 
    data['lag_price_t1'] = data.groupby(['id'])['sell_price'].transform(lambda x: x.shift(7)) 
    data['price_change_t1'] = (data['lag_price_t1'] - data['sell_price']) / (data['lag_price_t1']) 
    data['rolling_price_max_t365'] = data.groupby(['id'])['sell_price'].transform(lambda x: x.shift(1).rolling(365).max()) 
    data['price_change_t365'] = (data['rolling_price_max_t365'] - data['sell_price']) / (data['rolling_price_max_t365'])
    data['rolling_price_std_t7'] = data.groupby(['id'])['sell_price'].transform(lambda x: x.rolling(7).std())
    data['rolling_price_std_t30'] = data.groupby(['id'])['sell_price'].transform(lambda x: x.rolling(30).std())
    data.drop(['rolling_price_max_t365', 'lag_price_t1'], inplace = True, axis = 1)
    
    # time features
    data['date'] = pd.to_datetime(data['date'])
    data['week'] = data['date'].dt.week
    data['day'] = data['date'].dt.day
    data['dayofweek'] = data['date'].dt.dayofweek
    data['quarter'] = data['date'].dt.quarter
    
    # convert 'date' data type from datetime to integer
    data['date'] = pd.to_datetime(data['date']).dt.strftime('%Y%m%d')
    data['date'] = [int(strdt) for strdt in data['date']]
    
    return data

In [22]:
data = simple_fe(data)

Reduce memory for data set with new features so we can train the data

In [23]:
data = reduce_mem_usage(data)

Mem. usage decreased to 3342.33 Mb (45.6% reduction)


# XGBoost

This step is referenced from this notebook: https://www.kaggle.com/jestelrod/m5-eda-plotly-lstm-neural-network-vs-xgboost

In [24]:
import xgboost as xgb
from xgboost import plot_importance, plot_tree
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import GridSearchCV

Select all variables in data to be features except `demand` since we are training base on time splits and the `demand` feature used are rolling features (they are extracted from the past). Including this feature in features might give the actual target variable to the model but it will also cause data-leakage.

In [26]:
data.columns

Index(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'demand',
       'date', 'month', 'year', 'event_name_1', 'event_type_1', 'event_name_2',
       'event_type_2', 'snap_CA', 'snap_TX', 'snap_WI', 'sell_price',
       'lag_t28', 'lag_t29', 'lag_t30', 'rolling_mean_t7', 'rolling_std_t7',
       'rolling_mean_t30', 'rolling_mean_t90', 'rolling_mean_t180',
       'rolling_std_t30', 'rolling_skew_t30', 'rolling_kurt_t30',
       'price_change_t1', 'price_change_t365', 'rolling_price_std_t7',
       'rolling_price_std_t30', 'week', 'day', 'dayofweek', 'quarter'],
      dtype='object')

In [27]:
# target and label encoder features
features = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'date', 'month', 'year', 'event_name_1',
            'event_type_1', 'event_name_2', 'event_type_2', 'snap_CA', 'snap_TX', 'snap_WI', 'sell_price', 'lag_t28',
            'lag_t29', 'lag_t30', 'rolling_mean_t7', 'rolling_std_t7', 'rolling_mean_t30', 'rolling_mean_t90', 
            'rolling_mean_t180', 'rolling_std_t30', 'rolling_skew_t30', 'rolling_kurt_t30', 'price_change_t1', 
            'price_change_t365', 'rolling_price_std_t7', 'rolling_price_std_t30', 'week', 'day', 'dayofweek', 
            'quarter']

To train an XGBoost classifier, we split data to the training(from 2013 to 2015), validation set(from 2015 to 2016) and test set (last 28 days).
The training and validation set are converted from Pandas DataFrames into SVMLight format. To avoid overfitting, the `early_stopping_rounds` parameter is used to stop the training process after the test area under the curve (AUC) statistic fails to increase for 20 iterations.

In [28]:
def run_xgb(data):
    
    #XGBoost uses SVMLight data structure, not Numpy arrays or Pandas DataFrames 
    #training set
    x_train = data[data['date'] <= 20150424]
    y_train = x_train['demand']
    del x_train['demand']
    # validation set
    x_val = data[(data['date'] > 20150424) & (data['date'] < 20160522)]
    y_val = x_val['demand']
    del x_val['demand']
    
    #del data
    gc.collect()
    
    dtrain = xgb.DMatrix(x_train, y_train)
    dval = xgb.DMatrix(x_val, y_val)
    # used to calibrate predictions to mean of y 
    #base_y = y_train.mean()
    
    del x_train, y_train
    
    params = {
        'objective': 'count:poisson',          
        'booster': 'gbtree',                        # base learner will be decision tree
        'eval_metric': 'rmse',                      
        'eta': 0.075,                                 # learning rate
        'subsample': 0.75,                           # use 90% of rows in each decision tree
        'colsample_bytree': 0.9,                    # use 90% of columns in each decision tree
        #'max_depth': 15,                            # allow decision trees to grow to depth of 15
        #'base_score': base_y,                       # calibrate predictions to mean of y 
        'seed': 222                                 # set random seed for reproducibility
}
    
    
    watchlist = [(dtrain, 'train'), (dval, 'val')]
    
    # train model
    xgb_tg_encode = xgb.train(params,                   # set tuning parameters from above                   
                          dtrain,                   # training data
                          1000,                     # maximum of 1000 iterations (trees)
                          evals=watchlist,          # use watchlist for early stopping 
                          early_stopping_rounds=20,  # stop after 20 iterations (trees) without increase in AUC
                          verbose_eval=True)        # display iteration progress
         

    return x_val, xgb_tg_encode

In [30]:
# eta = 0.075 (slow xgb training)
validation, xgb_tg_encode = run_xgb(data)

[02:46:34] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1588600955503/work/src/gbm/gbtree.cc:138: Tree method is automatically selected to be 'approx' for faster speed. To use old behavior (exact greedy algorithm on single machine), set tree_method to 'exact'.
[0]	train-rmse:3.70773	val-rmse:3.68110
Multiple eval metrics have been passed: 'val-rmse' will be used for early stopping.

Will train until val-rmse hasn't improved in 20 rounds.
[1]	train-rmse:3.70328	val-rmse:3.67610
[2]	train-rmse:3.69730	val-rmse:3.66924
[3]	train-rmse:3.69073	val-rmse:3.66231
[4]	train-rmse:3.68437	val-rmse:3.65566
[5]	train-rmse:3.67614	val-rmse:3.64671
[6]	train-rmse:3.67123	val-rmse:3.64161
[7]	train-rmse:3.66446	val-rmse:3.63389
[8]	train-rmse:3.65540	val-rmse:3.62475
[9]	train-rmse:3.64782	val-rmse:3.61666
[10]	train-rmse:3.63885	val-rmse:3.60716
[11]	train-rmse:3.63031	val-rmse:3.59746
[12]	train-rmse:3.62280	val-rmse:3.58835
[13]	train-rmse:3.61389	val-rmse:3.57890
[14]	train-rmse:3.6

In [31]:
# Save the xgb model
import pickle
pickle.dump(xgb_tg_encode, open("xgb_tg_encode.dat", "wb"))

In [89]:
# Load saved xgb model 
xgb_tg_encode = pickle.load(open("xgb_tg_encode.dat", "rb"))

In [54]:
validation = data[(data['date'] > 20160424) & (data['date'] <= 20160522)]

In [50]:
test = data[data['date'] > 20160522]

In [ ]:
y_pred = xgb_tg_encode.predict(xgb.DMatrix(test, test['demand']))
test['pred_demand'] = y_pred

Define function `get_sub` to save the correct format for submissions file where we predict 28 forecast days (F1-F28) of items sold for each row.

In [ ]:
def get_sub(validation, test, submission):
    val_true = validation[['demand']]
    val_true = val_true.merge(id_date, left_index=True, right_index=True)
    val_true = pd.pivot(val_true, index = 'id', columns = 'date', values = 'demand').reset_index()
    
    val_true.columns = ['id'] + ['F' + str(i + 1) for i in range(28)]
    val_true['id'] = val_true['id'].str.replace('_evaluation','_validation')
    
    eval_pred = test[['pred_demand']]
    eval_pred = eval_pred.merge(id_date, left_index=True, right_index=True)
    eval_pred = pd.pivot(eval_pred, index = 'id', columns = 'date', values = 'pred_demand').reset_index()
    
    # change column name in consistent with sample_submission, like convert 20160523 to F1
    eval_pred.columns = ['id'] + ['F' + str(i + 1) for i in range(28)]
    
    final = pd.concat([val_true, eval_pred])
    
    # make sure the id order is the same as sample_submission
    final = submission[['id']].merge(final, on = 'id') 
    final.to_csv('xgb_submission.csv', index = False)
    return final

In [ ]:
final = get_sub(validation, test, submission)

In [ ]:
final.head()

In [ ]:
final.tail()